src/03_evaluation.py — Calibration, Risk Stratification, and Metrics

In [1]:
#1 Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#2 Set paths
import os
import pandas as pd
from sklearn.metrics import roc_auc_score, brier_score_loss, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.calibration import calibration_curve

In [3]:
#3
!pip install lifelines

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.3/117.3 kB 8.6 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4030 sha256=f311e2768f42af1ee371f663827e0069dcf0b318663c43a80b3f3957754e73f3
  Stored in directory: /root/.cache/pip/wheels/50/37/21/0a719b9d89c635e89ff24bd93b862882ad675279552013b2fb
Successfully built autograd-gamma


In [4]:
#4 Define paths
project_root = '/content/drive/MyDrive/fsgs_Kidney_Disease_study'
data_path = os.path.join(project_root, 'data', 'processed', 'fsgs_dataset_cleaned.csv')
output_path = os.path.join(project_root, 'outputs', 'evaluation')
os.makedirs(output_path, exist_ok=True)

In [6]:
#5 Load stratified dataset
df = pd.read_csv(data_path)

Assume 'risk_score' and 'risk_group' already exist from modeling step. If not, load model and compute them here

In [ ]:
#6. Calibration Curve

def plot_calibration_curve(y_true, y_prob, n_bins=10):
    prob_true, prob_pred = calibration_curve(y_true, y_prob, n_bins=n_bins)
    plt.figure(figsize=(8, 6))
    plt.plot(prob_pred, prob_true, marker='o', label='Calibration Curve')
    plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Perfect Calibration')
    plt.xlabel('Predicted Probability')
    plt.ylabel('Observed Probability')
    plt.title('Calibration Curve')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.savefig(os.path.join(output_path, 'calibration_curve.png'))
    plt.show()

plot_calibration_curve(df['outcome_dialysis_y_n'], df['risk_score'])

In [7]:
#7. Risk Stratification Summary

def summarize_risk_groups(df, time_col='survival_time', event_col='event'):
    summary = df.groupby('risk_group', observed=False)[[time_col, event_col]].agg(['count', 'mean', 'median'])
    print("Risk Group Summary:")
    print(summary)
    summary.to_csv(os.path.join(output_path, 'risk_group_summary.csv'))

# If 'risk_group' column does not exist, create it.
# Assuming 'FU_ESRD' can be used as a proxy for risk score for stratification.
if 'risk_group' not in df.columns:
    # Creating 4 risk groups based on quartiles of 'FU_ESRD'
    df['risk_group'] = pd.qcut(df['FU_ESRD'], q=4, labels=['Q1', 'Q2', 'Q3', 'Q4'])

# Call the function with appropriate column names from the loaded dataframe
# Assuming 'FU_ESRD' is the time to event and 'outcome_dialysis_y_n' is the event indicator.
summarize_risk_groups(df, time_col='FU_ESRD', event_col='outcome_dialysis_y_n')

Risk Group Summary:
           FU_ESRD                   outcome_dialysis_y_n                 
             count       mean median                count      mean median
risk_group                                                                
Q1             147   2.783401   2.81                  147  0.265306    0.0
Q2             220   6.458636   6.83                  220  0.104545    0.0
Q3              67   8.160299   8.08                   67  0.149254    0.0
Q4             145  14.488069  13.75                  145  0.110345    0.0


In [8]:
#8 Metrics: Concordance Index, AUC, Brier Score

from lifelines.utils import concordance_index
from sklearn.metrics import roc_auc_score, brier_score_loss

def evaluate_model(df, y_true_col, y_time_col, risk_score_col):
    y_true = df[y_true_col]
    y_time = df[y_time_col]
    risk_scores = df[risk_score_col]

    # Concordance Index
    c_index = concordance_index(y_time, -risk_scores, y_true)
    print(f"Concordance Index: {c_index:.3f}")

    # AUC (if binary classification)
    auc = roc_auc_score(y_true, risk_scores)
    print(f"AUC: {auc:.3f}")

    # Brier Score
    brier = brier_score_loss(y_true, risk_scores)
    print(f"Brier Score: {brier:.3f}")

    return c_index, auc, brier

In [9]:
#9 Create a dummy risk_score for demonstration (replace with actual model predictions in a real scenario)
# Inverting and scaling FU_ESRD: lower FU_ESRD means higher risk, so we want higher risk_score.
df['risk_score'] = 1 / (1 + df['FU_ESRD'])

# Call the evaluate_model function
c_index, auc, brier = evaluate_model(df, y_true_col='outcome_dialysis_y_n', y_time_col='FU_ESRD', risk_score_col='risk_score')

print(f"\nEvaluation Metrics using dummy risk scores:")
print(f"  Concordance Index: {c_index:.3f}")
print(f"  AUC: {auc:.3f}")
print(f"  Brier Score: {brier:.3f}")

Concordance Index: 1.000
AUC: 0.644
Brier Score: 0.129

Evaluation Metrics using dummy risk scores:
  Concordance Index: 1.000
  AUC: 0.644
  Brier Score: 0.129
